<a href="https://colab.research.google.com/github/noname1711/cryptography_theory/blob/main/AES_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pycryptodome

In [2]:
import numpy as np
from PIL import Image
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
from google.colab import files

In [3]:
def upload_image():
    uploaded = files.upload()
    for filename in uploaded.keys():
        print(f"Đã tải lên: {filename}")
        return filename

In [4]:
# Hàm đọc ảnh và chuyển sang numpy array
def image_to_array(image_path):
    image = Image.open(image_path)
    return np.array(image), image.mode

In [5]:
# Hàm chuyển numpy array sang ảnh
def array_to_image(array, mode, output_path):
    image = Image.fromarray(array, mode)
    image.save(output_path)

In [6]:
def encrypt_image_aes(image_array, key):
    cipher = AES.new(key, AES.MODE_ECB)
    flat_image = image_array.flatten()
    padded_data = pad(flat_image.tobytes(), AES.block_size)
    encrypted_data = cipher.encrypt(padded_data)
    encrypted_array = np.frombuffer(encrypted_data, dtype=np.uint8)
    sqrt_size = int(np.ceil(np.sqrt(len(encrypted_array))))
    padded_array = np.pad(encrypted_array, (0, sqrt_size**2 - len(encrypted_array)), mode='constant', constant_values=0)
    return padded_array.reshape((sqrt_size, sqrt_size)), len(flat_image)

In [7]:
def decrypt_image_aes(encrypted_array, key, original_shape, data_length):
    cipher = AES.new(key, AES.MODE_ECB)
    flat_encrypted = encrypted_array.flatten()
    decrypted_data = unpad(cipher.decrypt(flat_encrypted.tobytes()), AES.block_size)
    return np.frombuffer(decrypted_data[:data_length], dtype=np.uint8).reshape(original_shape)

In [8]:
print("1. Mã hóa hình ảnh bằng AES")
print("2. Giải mã hình ảnh bằng AES")
choice = input("Nhập 1 để mã hóa hoặc 2 để giải mã: ")

if choice == "1":
    image_path = upload_image()
    image_array, mode = image_to_array(image_path)
    original_shape = image_array.shape

    key = input("Nhập khóa AES (16 byte): ").encode()
    if len(key) != 16:
        print("Khóa AES phải có độ dài 16 byte.")
        exit()

    encrypted_array, data_length = encrypt_image_aes(image_array, key)
    encrypted_image_path = f"encrypted_AES_{image_path}"
    array_to_image(encrypted_array, "L", encrypted_image_path)
    print(f"Đã mã hóa hình ảnh và lưu tại: {encrypted_image_path}")
    files.download(encrypted_image_path)

elif choice == "2":
    encrypted_image_path = upload_image()
    encrypted_array, mode = image_to_array(encrypted_image_path)

    original_shape = eval(input("Nhập lại hình dạng gốc (VD: (1080, 720, 3)): "))
    data_length = int(input("Nhập lại số lượng pixel thực tế: "))
    key = input("Nhập khóa AES (16 byte): ").encode()
    if len(key) != 16:
        print("Khóa AES phải có độ dài 16 byte.")
        exit()

    decrypted_array = decrypt_image_aes(encrypted_array, key, original_shape, data_length)
    decrypted_image_path = f"decrypted_AES_{encrypted_image_path}"
    array_to_image(decrypted_array, mode, decrypted_image_path)
    print(f"Đã giải mã hình ảnh và lưu tại: {decrypted_image_path}")
    files.download(decrypted_image_path)

1. Mã hóa hình ảnh bằng AES
2. Giải mã hình ảnh bằng AES
Nhập 1 để mã hóa hoặc 2 để giải mã: 1


Saving logo-bach-khoa-ha-noi.jpg to logo-bach-khoa-ha-noi.jpg
Đã tải lên: logo-bach-khoa-ha-noi.jpg
Nhập khóa AES (16 byte): abRcj09iof123abc


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>